<a href="https://colab.research.google.com/github/lutakrystal305/DL_basic/blob/main/FeatureExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/nttuan8/DL_Tutorial

fatal: destination path 'DL_Tutorial' already exists and is not an empty directory.


In [ ]:
%cd /content/
!unzip /content/DL_Tutorial/L9/dataset.zip


In [ ]:
!pwd


/content


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications.vgg16 import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import random
import os


In [ ]:
#lấy các đường dẫn ảnh
image_path = list(paths.list_images('dataset/'))

#xáo trộn các đường dẫn
random.shuffle(image_path)

#imagepath: 'dataset/label/img.jpg',...
labels = [p.split(os.path.sep)[-2] for p in image_path]

#chuyển tên các label thành số
le = LabelEncoder()
labels = le.fit_transform(labels)



In [ ]:
#load vgg16  bỏ phần fully connected layer
model = VGG16(weights='imagenet', include_top=False)

#load ảnh và resize về kích thước của VGG
list_image = []
for (j, imagePath) in enumerate(image_path):

    image = load_img(imagePath, target_size=(224, 224))   #print(image) <PIL.Image.Image image mode=RGB size=224x224 at 0x7F527B0EB950> ....
    image = img_to_array(image)
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)

    list_image.append(image)

list_image = np.vstack(list_image)

#dùng pretrain model để lấy ra các đặc trưng của ảnh:
features = model.predict(list_image)
print(features.shape)

#same same flaten
features = features.reshape((features.shape[0], 512*7*7))
print(features.shape)


(1360, 7, 7, 512)
(1360, 25088)


In [ ]:
#chia train test set ti le 80:20
train_X, test_X, train_Y, test_Y = train_test_split(features, labels, test_size=0.2, random_state=42)
print(train_X.shape, train_Y.shape)

(1088, 25088) (1088,)


In [ ]:

# Grid search để tìm các parameter tốt nhất cho model. C = 1/lamda, hệ số trong regularisation. Solver là kiểu optimize
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
params = {'C' : [0.1, 1.0, 10.0, 100.0]}
#model = GridSearchCV(LogisticRegression(solver='lbfgs', multi_class='multinomial'), params)
model = GridSearchCV(LogisticRegression(), params)
model.fit(train_X, train_Y)
print('Best parameter for the model {}'.format(model.best_params_))

Best parameter for the model {'C': 10.0}


In [ ]:

# Đánh giá model
preds = model.predict(test_X)
print(classification_report(test_Y, preds))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94        17
           1       1.00      0.86      0.92        21
           2       1.00      0.92      0.96        12
           3       0.78      0.90      0.84        20
           4       0.89      0.94      0.92        18
           5       0.93      0.78      0.85        18
           6       1.00      1.00      1.00        11
           7       1.00      1.00      1.00        15
           8       1.00      1.00      1.00        17
           9       0.90      0.95      0.92        19
          10       0.82      0.93      0.87        15
          11       1.00      1.00      1.00        11
          12       0.82      0.90      0.86        20
          13       1.00      1.00      1.00        15
          14       1.00      1.00      1.00        12
          15       0.94      0.83      0.88        18
          16       0.92      0.92      0.92        13

    accuracy              